In [ ]:
%load_ext autoreload
%autoreload 2
import os

os.environ["PREFECT__FLOWS__CHECKPOINTING"] = "True"

import fsspec
import pandas as pd
import xarray as xr
import numpy as np
import cmip6_downscaling
import numpy as np
from xarray_schema import DataArraySchema, DatasetSchema

from cmip6_downscaling.data.observations import open_era5
from cmip6_downscaling.data.cmip import get_gcm, load_cmip
from dask.distributed import Client, LocalCluster
import warnings
warnings.filterwarnings("ignore")
import pytest
from cmip6_downscaling.analysis import analysis, metrics
from cmip6_downscaling.analysis.analysis import qaqc_checks, grab_top_city_data, load_top_cities, get_seasonal, change_ds
from cmip6_downscaling.analysis.plot import plot_cdfs, plot_values_and_difference, plot_seasonal
from cmip6_downscaling.analysis.qaqc import make_qaqc_ds
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from carbonplan import styles

styles.mpl.set_theme(style='carbonplan_light')


In [ ]:
# Pass as a parameter to be used by papermill: the runid which will be used to create the template for the raw and summary files
run_id = "MIROC6-ssp370-1990-1990-2090-2090-55-60-80-85-tasmax"
var = "tasmax"
gcm = "MIROC6"
scenario = "ssp370"
train_period_start = "1990"
train_period_end = "1990"
predict_period_start = "1990"
predict_period_end = "1990"

In [ ]:
print(run_id)

In [ ]:
plt.plot(range(0, 4), range(0, 4))